In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_parquet('favorite-songs.parquet')
df.head()

,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,20yLo8tCAM1LXixAdBf3f2,70,0.760,0.803,8,-2.867,0,0.0306,0.53100,0.000000,0.0555,0.922,136.945,audio_features,spotify:track:20yLo8tCAM1LXixAdBf3f2,https://api.spotify.com/v1/tracks/20yLo8tCAM1L...,https://api.spotify.com/v1/audio-analysis/20yL...,223074,4
1,1zsPaEkglFvxjAhrM8yhpr,91,0.901,0.589,1,-6.713,0,0.2890,0.00302,0.000237,0.0992,0.399,98.002,audio_features,spotify:track:1zsPaEkglFvxjAhrM8yhpr,https://api.spotify.com/v1/tracks/1zsPaEkglFvx...,https://api.spotify.com/v1/audio-analysis/1zsP...,196600,4
2,4ZlHgEGwZb3PCq0OWFbhcO,52,0.884,0.694,0,-3.241,1,0.0716,0.15600,0.018800,0.3000,0.597,93.025,audio_features,spotify:track:4ZlHgEGwZb3PCq0OWFbhcO,https://api.spotify.com/v1/tracks/4ZlHgEGwZb3P...,https://api.spotify.com/v1/audio-analysis/4ZlH...,188533,4
3,6G12ZafqofSq7YtrMqUm76,87,0.873,0.695,0,-4.207,0,0.0459,0.41900,0.000001,0.2110,0.823,131.963,audio_features,spotify:track:6G12ZafqofSq7YtrMqUm76,https://api.spotify.com/v1/tracks/6G12ZafqofSq...,https://api.spotify.com/v1/audio-analysis/6G12...,158805,4
4,4X3CV9rXo3hQrkb0fzRAux,59,0.666,0.784,8,-4.524,1,0.0342,0.15200,0.000003,0.0793,0.477,91.999,audio_features,spotify:track:4X3CV9rXo3hQrkb0fzRAux,https://api.spotify.com/v1/tracks/4X3CV9rXo3hQ...,https://api.spotify.com/v1/audio-analysis/4X3C...,247973,4


In [13]:
float_columns = df.select_dtypes(include=['float64']).columns.values
float_columns = list(float_columns)
for column in ['popularity', 'duration_ms', 'time_signature']:
  float_columns.append(column)

In [20]:
scaler = StandardScaler()

unstandardized_df = df[float_columns].copy()
standardized = scaler.fit_transform(unstandardized_df)

array([[ 0.58630059,  0.99088281,  1.23962078, ...,  0.76039674,
         0.429885  ,  0.30048116],
       [ 1.3704329 ,  0.16331723,  0.4431757 , ...,  1.87646292,
         0.02553983,  0.30048116],
       [ 1.27589212,  0.56936576,  1.16217136, ..., -0.19623142,
        -0.09766983,  0.30048116],
       ...,
       [ 0.77538214,  0.68537963,  0.44462529, ..., -0.56825348,
        -0.35951517,  0.30048116],
       [ 0.41946392, -0.87694042, -0.31309508, ..., -0.7276915 ,
         0.30310139,  0.30048116],
       [ 0.97002491,  0.6815125 ,  0.33093617, ..., -1.09971356,
        -0.35914861,  0.30048116]])

In [24]:
# put the standardized data back into a dataframe, as well as the `id` column back
standardized_df = pd.DataFrame(standardized, columns=float_columns)
standardized_df['id'] = df['id']
standardized_df = standardized_df[['id'] + float_columns]
standardized_df.head()


,id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,popularity,duration_ms,time_signature
0,20yLo8tCAM1LXixAdBf3f2,0.586301,0.990883,1.239621,-0.768237,0.368567,-0.563326,-1.015789,2.115254,0.760272,0.760397,0.429885,0.300481
1,1zsPaEkglFvxjAhrM8yhpr,1.370433,0.163317,0.443176,2.406790,-1.205910,-0.562588,-0.617245,-0.182452,-0.598943,1.876463,0.025540,0.300481
2,4ZlHgEGwZb3PCq0OWFbhcO,1.275892,0.569366,1.162171,-0.264460,-0.749712,-0.504827,1.214050,0.687425,-0.772653,-0.196231,-0.097670,0.300481
3,6G12ZafqofSq7YtrMqUm76,1.214719,0.573233,0.962128,-0.580242,0.034575,-0.563322,0.402371,1.680316,0.586386,1.663879,-0.551714,0.300481
4,4X3CV9rXo3hQrkb0fzRAux,0.063546,0.917407,0.896483,-0.724003,-0.761640,-0.563315,-0.798733,0.160227,-0.808464,0.175791,0.810175,0.300481
